In [1]:
#kodingan ini hanya memanfaatkan image Part 1-2 dari situs AffectNet
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
import numpy as np

In [3]:
import os
import cv2
import pandas as pd

In [4]:
label_root = 'training_fixed.csv'
image_root = 'Affectnet/Manually_Annotated_Images'

In [5]:
label_df = pd.read_csv(label_root)
image_dirs = [image_root + '/' + dir_ for dir_ in label_df['subDirectory_filePath'].tolist()]
labels = [x for x in label_df['expression'].tolist()]

In [6]:
H = 128
W = 128
def read_image(file_path):
    global H,W
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    dimension = (H,W)
    resized = cv2.resize(img, dimension)
    return resized
def prep_data(images):
    global H,W
    label = []
    data = np.ndarray((10000, 1, H, W), dtype=np.uint8)
    i = 0
    for image_file in images:
        if(i == 10000):
            break
        try:
            image = read_image(image_file)
        except:
            continue
        data[i] = image
        label.append(labels[i])
        i += 1
        if i%50 == 0: print('{} image(s) have been read'.format(i))
    return data, np.array(label)
images_fix, labels_fix = prep_data(image_dirs)

50 image(s) have been read
100 image(s) have been read
150 image(s) have been read
200 image(s) have been read
250 image(s) have been read
300 image(s) have been read
350 image(s) have been read
400 image(s) have been read
450 image(s) have been read
500 image(s) have been read
550 image(s) have been read
600 image(s) have been read
650 image(s) have been read
700 image(s) have been read
750 image(s) have been read
800 image(s) have been read
850 image(s) have been read
900 image(s) have been read
950 image(s) have been read
1000 image(s) have been read
1050 image(s) have been read
1100 image(s) have been read
1150 image(s) have been read
1200 image(s) have been read
1250 image(s) have been read
1300 image(s) have been read
1350 image(s) have been read
1400 image(s) have been read
1450 image(s) have been read
1500 image(s) have been read
1550 image(s) have been read
1600 image(s) have been read
1650 image(s) have been read
1700 image(s) have been read
1750 image(s) have been read
1800 

In [19]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
 
X_train, X_test, y_train, y_test = train_test_split(images_fix, labels_fix, test_size=0.20, random_state=69)

X_train = X_train.reshape(X_train.shape[0], H, W, 1)
X_test = X_test.reshape(X_test.shape[0], H, W, 1)

In [20]:
def CNN(conv1_units, conv2_units, kernel_size, pool_size, dropout_rate):
    model = Sequential()
    model.add(Conv2D(conv1_units, kernel_size=(kernel_size, kernel_size),
                     activation='relu',
                     input_shape=(128,128,1)))
    model.add(Conv2D(conv2_units, (kernel_size, kernel_size), activation='relu'))
    model.add(MaxPooling2D(pool_size=(pool_size,pool_size)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(2*dropout_rate))
    model.add(Dense(11, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='Adadelta',metrics=['accuracy'])
    return model

In [21]:
model_1 = CNN(48,56,3,4,0.2)

In [ ]:
model_1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10,batch_size=10, verbose=2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
 - 853s - loss: 11.0815 - acc: 0.3121 - val_loss: 10.5412 - val_acc: 0.3460
Epoch 2/10
